# Download

In [1]:
import pandas as pd
import requests
from src.downloading_and_cleaning import download_data, clean_pokemon_csv_dataframe
from src.enriching import get_evol_trigger, compare_and_add_to_DataFrame, get_game, take_region, get_region
#first import libraries

In [40]:
download_data() #download th data

Write the URL from dataset you wanto to download: https://www.kaggle.com/abcsds/pokemon
Download file
Descompressing files
removing .zip
Moving file to data


'Your file: pokemon.csv is in data folder. Enjoy!!'

In [5]:
#Read the pokemon.csv
pokemon = pd.read_csv("data/pokemon.csv")

In [6]:
pokemon.head()#explore the Dataframe

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [7]:
pokemon.dtypes

#              int64
Name          object
Type 1        object
Type 2        object
Total          int64
HP             int64
Attack         int64
Defense        int64
Sp. Atk        int64
Sp. Def        int64
Speed          int64
Generation     int64
Legendary       bool
dtype: object

# Clean

In [41]:
pokemon = clean_pokemon_csv_dataframe() #clean the dataframe
pokemon
#In the cleanup we have removed the "megas" pokemon from the "Name" column, 
#have removed the "Total" column and the "Legendaries" column. And have set "#" as index.

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
#,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1
4,Charmander,Fire,NaN,39,52,43,60,50,65,1
5,Charmeleon,Fire,NaN,58,64,58,80,65,80,1
...,...,...,...,...,...,...,...,...,...,...
711,GourgeistSuper Size,Ghost,Grass,85,100,122,58,75,54,6
712,Bergmite,Ice,NaN,55,69,85,32,35,28,6
713,Avalugg,Ice,NaN,95,117,184,44,46,28,6


# Enriching

Let's enrich the dataframe with two columns that will tell us more about the pokemon. The first one about how they can evolve, and the second one about what region they are from. 
Will do this by calling two API's.

In [9]:
response = requests.get("https://pokeapi.co/api/v2/evolution-trigger/")#call API

In [10]:
response.json()#check the json

{'count': 5,
 'next': None,
 'previous': None,
 'results': [{'name': 'level-up',
   'url': 'https://pokeapi.co/api/v2/evolution-trigger/1/'},
  {'name': 'trade', 'url': 'https://pokeapi.co/api/v2/evolution-trigger/2/'},
  {'name': 'use-item',
   'url': 'https://pokeapi.co/api/v2/evolution-trigger/3/'},
  {'name': 'shed', 'url': 'https://pokeapi.co/api/v2/evolution-trigger/4/'},
  {'name': 'other', 'url': 'https://pokeapi.co/api/v2/evolution-trigger/5/'}]}

In [11]:
level_up = get_evol_trigger("https://pokeapi.co/api/v2/evolution-trigger/1/")
trade = get_evol_trigger("https://pokeapi.co/api/v2/evolution-trigger/2/")
item = get_evol_trigger("https://pokeapi.co/api/v2/evolution-trigger/3/")
shed = get_evol_trigger("https://pokeapi.co/api/v2/evolution-trigger/4/")
others = get_evol_trigger("https://pokeapi.co/api/v2/evolution-trigger/5/")
#take a list of each pokemon in each evolutionary method.
#the function calls the Api of each one and returns a list of information

In [12]:
pokemon["Evolution"] = pokemon["Name"].apply(lambda x: compare_and_add_to_DataFrame(level_up, trade, item, shed, others, x))
#by means of .apply + lambda the column is included in the dataframe

In [13]:
pokemon

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Evolution
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,Base
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,Level_up
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,Level_up
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,Base
5,Charmeleon,Fire,NaN,58,64,58,80,65,80,1,Level_up
...,...,...,...,...,...,...,...,...,...,...,...
711,GourgeistSuper Size,Ghost,Grass,85,100,122,58,75,54,6,Base
712,Bergmite,Ice,NaN,55,69,85,32,35,28,6,Base
713,Avalugg,Ice,NaN,95,117,184,44,46,28,6,Level_up


In [24]:
regions = requests.get("https://pokeapi.co/api/v2/region/").json()#call the API

In [25]:
region = take_region("https://pokeapi.co/api/v2/region/")#"take the region" calls the Api and returns a list of region names

In [26]:
pokemon["Region"] = pokemon["Generation"].apply(lambda x: get_region(x, region))
#add the info to the DataFrame

In [27]:
pokemon

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Evolution,Region
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,Base,kanto
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,Level_up,kanto
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,Level_up,kanto
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,Base,kanto
5,Charmeleon,Fire,NaN,58,64,58,80,65,80,1,Level_up,kanto
...,...,...,...,...,...,...,...,...,...,...,...,...
711,GourgeistSuper Size,Ghost,Grass,85,100,122,58,75,54,6,Base,kalos
712,Bergmite,Ice,NaN,55,69,85,32,35,28,6,Base,kalos
713,Avalugg,Ice,NaN,95,117,184,44,46,28,6,Level_up,kalos


In [ ]:
pokemon.to_csv('poks.csv')

# Bonus - Pokemon Games

Here, we call another api to get the pokemon games according to their generation. And we classify them according to the region where their history is located.

In [14]:
response = requests.get("https://pokeapi.co/api/v2/generation/")

In [15]:
response.json()

{'count': 8,
 'next': None,
 'previous': None,
 'results': [{'name': 'generation-i',
   'url': 'https://pokeapi.co/api/v2/generation/1/'},
  {'name': 'generation-ii', 'url': 'https://pokeapi.co/api/v2/generation/2/'},
  {'name': 'generation-iii', 'url': 'https://pokeapi.co/api/v2/generation/3/'},
  {'name': 'generation-iv', 'url': 'https://pokeapi.co/api/v2/generation/4/'},
  {'name': 'generation-v', 'url': 'https://pokeapi.co/api/v2/generation/5/'},
  {'name': 'generation-vi', 'url': 'https://pokeapi.co/api/v2/generation/6/'},
  {'name': 'generation-vii', 'url': 'https://pokeapi.co/api/v2/generation/7/'},
  {'name': 'generation-viii',
   'url': 'https://pokeapi.co/api/v2/generation/8/'}]}

In [29]:
gen1 = get_game("https://pokeapi.co/api/v2/generation/1/")
gen2 = get_game("https://pokeapi.co/api/v2/generation/2/")
gen3 = get_game("https://pokeapi.co/api/v2/generation/3/")
gen4 = get_game("https://pokeapi.co/api/v2/generation/4/")
gen5 = get_game("https://pokeapi.co/api/v2/generation/5/")
gen6 = get_game("https://pokeapi.co/api/v2/generation/6/")

In [32]:
games = [gen1, gen2, gen3, gen4, gen5, gen6]

In [33]:
generations = pokemon["Region"].unique()

In [34]:
games = pd.DataFrame(games, generations)

In [35]:
games = games.drop(games[[3, 4]], axis=1)

In [36]:
games

,0,1,2
kanto,red-blue,yellow,None
johto,gold-silver,crystal,None
hoenn,ruby-sapphire,emerald,firered-leafgreen
sinnoh,diamond-pearl,platinum,heartgold-soulsilver
unova,black-white,black-2-white-2,None
kalos,x-y,omega-ruby-alpha-sapphire,None


In [37]:
games.columns = ["Original", "Remake1", "RemakePastGen"]

In [38]:
games

,Original,Remake1,RemakePastGen
kanto,red-blue,yellow,None
johto,gold-silver,crystal,None
hoenn,ruby-sapphire,emerald,firered-leafgreen
sinnoh,diamond-pearl,platinum,heartgold-soulsilver
unova,black-white,black-2-white-2,None
kalos,x-y,omega-ruby-alpha-sapphire,None
